In [298]:
import sys 
from nltk.probability import FreqDist,MLEProbDist
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
import string
import numpy as np
import math
import random

f=open("sherlock.txt")
f=f.read()

### Cleaning the dataset

In [299]:
tokenizer = RegexpTokenizer(r'[\w\?\.\!\'-]+')
tokens=tokenizer.tokenize(f)

printable = set(string.printable)

new_tokens = []
for i,t in enumerate(tokens):
    tokens[i] = filter(lambda x: x in printable , t)
    tokens[i]=tokens[i].replace("'",'')
    tokens[i]=tokens[i].replace("--",' ')
    if tokens[i] != '':
        new_tokens.append(tokens[i].lower())
    
corpus = ' '.join(new_tokens)

startsentences = sent_tokenize(corpus)
sentences=[]

### Adding < S >  and  < /S >

In [300]:
for sentence in startsentences:
    finalsentence = '<S> ' + sentence[:-1] + ' </S>'
    sentences.append(finalsentence)

In [301]:
#sentences

### Training and test

In [302]:
training = sentences[0:(len(sentences)*8/10)]
testing= sentences[(len(sentences)*8/10):]

# MLE

In [303]:
new_sentences =' '.join(sentences)
list_words = new_sentences.split(' ')
list_words.remove('')

###  Unigram MLE

In [304]:
oneDict= FreqDist(list_words)
unigrams = oneDict.keys()

In [305]:
#oneDict

In [306]:
oneMLE = MLEProbDist(oneDict)

In [307]:
unigramMLE={}
for key in oneDict:
    unigramMLE[key]=oneMLE.prob(key)
    

###  Bigram MLE

In [308]:
bigrams_list=[]
flag=0
for i in range(0,len(list_words)-1):
    if list_words[i]=="</S>" and list_words[i+1]=="<S>":
        flag=1
    else:
        bigrams_list.append((list_words[i],list_words[i+1]))

In [309]:
bigramDict = FreqDist(bigrams_list)

In [310]:
bigramMLE={}
for key in bigramDict:
    d = oneDict[key[0]]
    bigramMLE[key] = float(bigramDict[key])/d

###  Trigram MLE

In [311]:
trigrams_list=[]
flag=0
for i in range(0,len(list_words)-2):
    if ((list_words[i]=="</S>" and list_words[i+1]=="<S>") or 
        (list_words[i+1]=="</S>" and list_words[i+2]=="<S>")):
        flag=1
    else:
        trigrams_list.append((list_words[i],list_words[i+1], list_words[i+2]))
            

In [312]:
trigramDict = FreqDist(trigrams_list)

In [313]:
trigramMLE={}
for key in trigramDict:
    d = bigramDict[(key[0],key[1])]
    trigramMLE[key] = float(trigramDict[key])/d

### Quadgram MLE

In [314]:
quadgrams_list=[]
flag=0
for i in range(0,len(list_words)-3):
    if ((list_words[i]=="</S>" and list_words[i+1]=="<S>") or 
    (list_words[i+1]=="</S>" and list_words[i+2]=="<S>") or 
    (list_words[i+2]=="</S>" and list_words[i+3]=="<S>")):
        flag=1
    else:
        quadgrams_list.append((list_words[i],list_words[i+1], list_words[i+2] , list_words[i+3]))

In [315]:
quadgramDict = FreqDist(quadgrams_list)

In [316]:
quadgramMLE={}
for key in quadgramDict:
    d = trigramDict[(key[0],key[1],key[2])]
    quadgramMLE[key] = float(quadgramDict[key])/d

In [317]:
## Number of possible N-grams v/s actual number of N-grams in the corpus

# Unigrams 
print "\nNumber of possible unigrams : " , len(unigrams)
print "Number of actual unigrams in the corpus : " , len(unigrams)

# Bigrams
print "\nNumber of possible bigrams : " , (len(unigrams)*(len(unigrams) -1)/2)
print "Number of actual bigrams in the corpus : " , len(bigramDict.keys())

# Trigrams
print "\nNumber of possible trigrams : " , (len(unigrams)*(len(unigrams) -1)*(len(unigrams)-2)/6)
print "Number of actual trigrams in the corpus : " , len(trigramDict.keys())

# Quadgrams
print "\nNumber of possible quadgrams : " , (len(unigrams)*(len(unigrams) -1)*(len(unigrams)-2)*(len(unigrams)-3)/24)
print "Number of actual quadgrams in the corpus : " , len(quadgramDict.keys())


Number of possible unigrams :  8298
Number of actual unigrams in the corpus :  8298

Number of possible bigrams :  34424253
Number of actual bigrams in the corpus :  49315

Number of possible trigrams :  95194534296
Number of actual trigrams in the corpus :  84520

Number of possible quadgrams :  197409665496330
Number of actual quadgrams in the corpus :  93033


## Generator and Probability functions

### Generator function

In [318]:
#bigramMLE

In [319]:
def Generator(model_name):
    
    gensentence = ['<S>']
    gensenlength = 1
    madesentence = ''
    
    
    if model_name == 1:
        
        flag =1 

        while(flag==1):    
            nextword = np.random.choice(unigramMLE.keys(),1,p=unigramMLE.values())       
            if nextword[0] == '</S>':
                flag =0 
            if nextword[0] == '<S>':
                flag=1
            else:
                gensentence.append(nextword[0]) 

#########################################################################      
    if model_name == 2:
        flag=1
        while(flag==1):
            first={}
            prob= {}
            for bigram in (bigramMLE.keys()):            
                if bigram not in first.keys() and bigram[0]==gensentence[len(gensentence)-1]:

                    first[bigram] = bigramMLE[bigram]

            for bigram in first:
                prob[bigram[1]]= bigramMLE[bigram]

            nextword = np.random.choice(prob.keys(),1,p=prob.values())
            gensentence.append(nextword[0])
            
            if nextword[0] == '</S>':     
                flag=0

        
############################################################################
    if model_name == 3:
        flag =1
        
        firstfrombigram={}
        for bigram in (bigramMLE.keys()):            
            if bigram not in firstfrombigram.keys() and bigram[0]=='<S>':
                firstfrombigram[bigram[1]] = bigramMLE[bigram]
             
        nextword = np.random.choice(firstfrombigram.keys(),1,p=firstfrombigram.values())
        gensentence.append(nextword[0])
        
        
        while(flag==1):
            first={}
            prob= {}
            for trigram in (trigramMLE.keys()):            
                if trigram not in first.keys() and trigram[0]==gensentence[len(gensentence)-2] and trigram[1]==gensentence[len(gensentence)-1]:
                    first[trigram] = trigramMLE[trigram]

            for trigram in first:
                prob[trigram[2]]= trigramMLE[trigram]

            nextword = np.random.choice(prob.keys(),1,p=prob.values())
            gensentence.append(nextword[0])
            
            if nextword[0] == '</S>':
                flag=0
    
        
############################################################################
    if model_name == 4:
        flag =1
        
        firstfrombigram={}
        for bigram in (bigramMLE.keys()):            
            if (bigram not in firstfrombigram.keys() and 
                bigram[0]=='<S>'):
                firstfrombigram[bigram[1]] = bigramMLE[bigram]

        nextword = np.random.choice(firstfrombigram.keys(),1,p=firstfrombigram.values())
        gensentence.append(nextword[0])

        secondfromtrigram = {}
        for trigram in (trigramMLE.keys()):            
            if (trigram not in secondfromtrigram.keys() and 
                trigram[0]=='<S>' and trigram[1]==gensentence[1]):
                secondfromtrigram[trigram[2]] = trigramMLE[trigram]

        nextword = np.random.choice(secondfromtrigram.keys(),1,p=secondfromtrigram.values())
        gensentence.append(nextword[0])

        while(flag==1):
            first={}
            prob= {}
            for quadgram in (quadgramMLE.keys()):            
                if (quadgram not in first.keys() and 
                quadgram[0]==gensentence[len(gensentence)-3] and 
                quadgram[1]==gensentence[len(gensentence)-2] and 
                quadgram[2]==gensentence[len(gensentence)-1]):
                    
                    first[quadgram] = quadgramMLE[quadgram]


            for quadgram in first:
                prob[quadgram[3]]= quadgramMLE[quadgram]

            nextword = np.random.choice(prob.keys(),1,p=prob.values())
            gensentence.append(nextword[0])
            
            if nextword[0] == '</S>':
                flag=0


############################################################################
    
    madesentence = ' '.join(gensentence)            
    return madesentence
                
 

In [320]:
for i in range(0,5):
    print(Generator(4))

<S> well watson said holmes reaching up his hand though so he followed me there and he saw me without pa knowing anything about it </S>
<S> you must know said he that i am still in the dark to see what would come of this strange affair </S>
<S> he was a lawyer </S>
<S> this he opened and made a pile while poor frank here had a claim that petered out and came to nothing </S>
<S> in dress now for example </S>


In [321]:
for i in range(0,5):
    print(Generator(3))

<S> i confess that the status of my trunk turned out the contents and drew from the king </S>
<S> it was on board of a broad intelligent face sloping down to streatham since i saw the man who wrote it was something else which she must have almost gone and the ruffian who pursued me </S>
<S> what do you mean </S>
<S> you fail however to reason from what i wished to have taken it upstairs and a large scale and have this money or if they believed my statement for it was obvious from the little area of light now bright now faint as the old trick </S>
<S> i cried </S>


In [322]:
for i in range(0,5):
    print(Generator(2))

<S> inspector </S>
<S> and i retired to the afternoon </S>
<S> precisely i have seen a blind </S>
<S> my dear watson said holmes are not heard a hundred yards from america with laudanum in his hand closed the matter up my neighbours who are getting these german i waited in the ring </S>
<S> but have his life in the other consideration that so astute a demon ill state of the moonlight and benevolent curiosity </S>


In [323]:
for i in range(0,5):
    print(Generator(1))

<S> there holmes </S>
<S> to face little over forget i away the four need fortune who dear </S>
<S> a </S>
<S> me solemn the put i escape daring </S>
<S> he notes the i my ive so as reached that witness was but him band were is that read beauty </S>


### Probability Function

In [324]:
#bigramMLE

In [325]:
def Probability(sentence,model_name):
    sentence = sentence.lower()
    new_sentence = '<S> ' + sentence + ' </S>'
    listOFwords = new_sentence.split(' ')
    ProbINlog= 0
    
    
    if model_name ==1 : 
        
        flag = 1
        logprob=0
        for Index in range(0,len(listOFwords)):
            present = 0 
            one = listOFwords[Index]
            
            if one in unigramMLE:
                present = unigramMLE[one]
                logprob = logprob + math.log(present)
            else:
                flag = 0
                break
        if flag == 0 :
            print '\nThis sentence is not present in corpus'
        else : 
            ProbINlog = logprob
            print '\nThis sentence is present in corpus'
        
##########################################################################


    if model_name == 2 :
       
        flag = 1
        logprob=0
        for Index in range(0,len(listOFwords)-1):
            present = 0 
            one = listOFwords[Index]
            two = listOFwords[Index+1]
            onetwo=(one,two)
            
            if onetwo in bigramMLE:
                present = bigramMLE[onetwo]
                logprob = logprob + math.log(present)
            else:
                flag = 0
                break
        if flag == 0 :
            print '\nThis sentence is not present in corpus'
        else : 
            ProbINlog = logprob
            print '\nThis sentence is present in corpus'
            
            
##########################################################################    
    
    
    if model_name == 3 :
        #print(listOFwords)
        flag = 1
        logprob=0
        for Index in range(0,len(listOFwords)-2):
            present = 0 
            one = listOFwords[Index]
            two = listOFwords[Index+1]
            three = listOFwords[Index+2]
            onetwothree=(one,two,three)
            
            if onetwothree in trigramMLE:
                present = trigramMLE[onetwothree]
                logprob = logprob + math.log(present)
            else:
                flag = 0
                break
        if flag == 0 :
            print 'This sentence is not present in corpus'
        else : 
            ProbINlog = logprob
            print 'This sentence is present in corpus'
            
##########################################################################
    
    
    if model_name == 4:
        
        flag = 1
        logprob=0
        for Index in range(0,len(listOFwords)-3):
            present = 0 
            one = listOFwords[Index]
            two = listOFwords[Index+1]
            three = listOFwords[Index+2]
            four = listOFwords[Index+3]
            onetwothreefour=(one,two,three,four)
            
            if onetwothreefour in quadgramMLE:
                present = quadgramMLE[onetwothreefour]
                logprob = logprob + math.log(present)
            else:
                flag = 0
                break
        if flag == 0 :
            print '\nThis sentence is not present in corpus'
        else : 
            ProbINlog = logprob
            print '\nThis sentence is present in corpus'
##########################################################################
    
    
    if ProbINlog == 0 :
        return 0
    else:
        return math.exp(ProbINlog)
                
            
            
        

In [326]:
Probability('not a bit doctor',1)


This sentence is present in corpus


5.9044799151280424e-15

In [327]:
Probability('this assignment is really long',4)


This sentence is not present in corpus


0

## add-1 smoothing for bigram model 

In [328]:

def add_one(one,two):  # returns probability,effective count after add-1 smoothing 
    c = 0 
    add_1bigramProb = 0
    add_1effectivecount = 0
    dv = oneDict[one] + len(unigrams)
    d = oneDict[one] 
    
    if (one,two) in bigramDict.keys():        
        c=bigramDict[(one,two)]
        add_1bigramProb = (float(c + 1))/dv
    else:
        add_1bigramProb = (float(1))/dv
    
    add_1effectivecount= (add_1bigramProb*d)
    
    return [add_1bigramProb , add_1effectivecount]

In [329]:
add_one('eagerly', 'into')

[0.0002409058058299205, 0.000963623223319682]

In [330]:
add_one('and','the')

[0.017686593562079942, 53.23664662186063]

### Change in the count post add-1 smoothing


In [331]:
someexamples=[('dressing-gown', 'his'),('was','a'),('laying', 'his'), ('eagerly', 'into'),('shed', 'in'),('and','the')]


In [332]:
### Some examples are as follows : 

for i in someexamples:
    
    print '\nExample ',(someexamples.index(i)+1)
    print 'Bigram : ', i
    print 'Count of bigram in corpus : ', bigramDict[i]
    lst = add_one(i[0],i[1])
    print 'Count of bigram after add-1 smoothing : ', lst[1]
    print ' '


Example  1
Bigram :  ('dressing-gown', 'his')
Count of bigram in corpus :  1
Count of bigram after add-1 smoothing :  0.0014450867052
 

Example  2
Bigram :  ('was', 'a')
Count of bigram in corpus :  160
Count of bigram after add-1 smoothing :  23.3838071693
 

Example  3
Bigram :  ('laying', 'his')
Count of bigram in corpus :  1
Count of bigram after add-1 smoothing :  0.00168573148706
 

Example  4
Bigram :  ('eagerly', 'into')
Count of bigram in corpus :  1
Count of bigram after add-1 smoothing :  0.00096362322332
 

Example  5
Bigram :  ('shed', 'in')
Count of bigram in corpus :  1
Count of bigram after add-1 smoothing :  0.00024099289071
 

Example  6
Bigram :  ('and', 'the')
Count of bigram in corpus :  199
Count of bigram after add-1 smoothing :  53.2366466219
 


#### Drastic change can be seen in the counts of bigrams after add-1 smoothing because some of the probability mass of bigrams present in the corpus gets distributed to bigrams that are not present in the corpus. Add-1 smoothing is used to consider bigrams that can be formed from the unigrams present in the corpus but these bigrams do not originally exist in the corpus.

## Good-turing smoothing technique for bigrams

In [334]:
frequencyOFcounts={}
for i in bigramDict.values():
    if i not in frequencyOFcounts:
        frequencyOFcounts[i]=1
    else:
        frequencyOFcounts[i]= frequencyOFcounts[i] + 1
        

In [335]:
frequencyOFcounts        

{1: 36863,
 2: 5885,
 3: 2234,
 4: 1159,
 5: 671,
 6: 479,
 7: 296,
 8: 227,
 9: 203,
 10: 166,
 11: 135,
 12: 104,
 13: 86,
 14: 60,
 15: 45,
 16: 47,
 17: 46,
 18: 46,
 19: 36,
 20: 32,
 21: 27,
 22: 30,
 23: 22,
 24: 17,
 25: 14,
 26: 21,
 27: 14,
 28: 18,
 29: 18,
 30: 22,
 31: 12,
 32: 10,
 33: 11,
 34: 8,
 35: 9,
 36: 10,
 37: 16,
 38: 5,
 39: 9,
 40: 10,
 41: 7,
 42: 4,
 43: 7,
 44: 3,
 45: 5,
 46: 5,
 47: 5,
 48: 5,
 49: 1,
 50: 5,
 51: 2,
 52: 3,
 53: 6,
 54: 3,
 55: 2,
 56: 2,
 57: 2,
 59: 2,
 60: 2,
 61: 6,
 63: 4,
 65: 4,
 66: 2,
 68: 3,
 70: 3,
 71: 2,
 74: 4,
 75: 1,
 76: 1,
 77: 2,
 79: 1,
 80: 3,
 81: 1,
 82: 1,
 83: 2,
 84: 3,
 86: 2,
 88: 1,
 90: 2,
 92: 1,
 93: 1,
 95: 2,
 96: 1,
 97: 1,
 98: 1,
 99: 4,
 102: 1,
 103: 1,
 104: 1,
 106: 1,
 108: 1,
 109: 1,
 110: 3,
 114: 1,
 115: 1,
 116: 1,
 117: 1,
 118: 1,
 120: 2,
 122: 2,
 123: 1,
 125: 1,
 130: 2,
 131: 1,
 134: 1,
 135: 2,
 136: 1,
 138: 1,
 139: 1,
 140: 1,
 142: 1,
 143: 1,
 149: 1,
 151: 1,
 154: 1,
 158: 1

In [336]:
effectivecount1_10={}
freqbetween1_10 = frequencyOFcounts.values()[:12]

for i in range(0,10):
    p = float((i+2)*freqbetween1_10[i+1])/freqbetween1_10[i]
    effectivecount1_10[i+1]=p


In [337]:
lstd=[]
d_gt=0

for key in effectivecount1_10:
    lstd.append(key - effectivecount1_10[key])
    
d_gt = np.mean(lstd)

#### List of values of d for bigrams having original counts between 1 to 10 

In [338]:
lstd

[0.6807096546672815,
 0.8611724723874257,
 0.9247985675917638,
 1.1052631578947367,
 0.7168405365126675,
 1.674321503131524,
 0.8648648648648649,
 -0.04845814977973539,
 0.8226600985221673,
 1.0542168674698793]

#### Effective value of d

In [339]:
d_gt

0.8656389573262576

In [340]:
#list_words

In [341]:
def good_turing(one,two):
    c=0
    gt_effectivecount=0
    gt_bigramProb=0
    d = oneDict[one] 
    
    if (one,two) in bigramDict.keys():
        c=bigramDict[(one,two)]
        gt_bigramProb = float(c - d_gt)/d
    else:
        gt_bigramProb = float(frequencyOFcounts[1])/len(list_words)
    return gt_bigramProb
           

In [342]:
good_turing('and','the')

0.06582536911716735

# Perplexity

## Training 

In [343]:
train =' '.join(training)
trainwords = train.split(' ')
trainwords.remove('')

In [345]:
# Unigram counts
unigramDictTrain= FreqDist(trainwords)
unigramsTrain = unigramDictTrain.keys()

In [346]:
# Bigram counts 
bigramlistTrain=[]
flag=0
for i in range(0,len(trainwords)-1):
    if trainwords[i]=="</S>" and trainwords[i+1]=="<S>":
        flag=1
    else:
        bigramlistTrain.append((trainwords[i],trainwords[i+1]))

bigramDictTrain = FreqDist(bigramlistTrain)

### Add-one

In [347]:
def add_oneTrain(one,two):  # returns probability,effective count after add-1 smoothing 
    c = 0 
    add_1bigramProb = 0
    
    dv = unigramDictTrain[one] + len(unigramsTrain)
    d = unigramDictTrain[one] 
    
    if (one,two) in bigramDictTrain.keys():        
        c=bigramDictTrain[(one,two)]
        add_1bigramProb = (float(c + 1))/dv
    else:
        add_1bigramProb = (float(1))/dv
    
    return add_1bigramProb 

In [348]:
add_oneTrain('sherlock','is')

0.00013157894736842105

### Good Turing

In [349]:
frequencyOFcountsTrain={}
for i in bigramDictTrain.values():
    if i not in frequencyOFcountsTrain:
        frequencyOFcountsTrain[i]=1
    else:
        frequencyOFcountsTrain[i]= frequencyOFcountsTrain[i] + 1

In [351]:
frequencyOFcountsTrain

{1: 31217,
 2: 4917,
 3: 1822,
 4: 902,
 5: 505,
 6: 372,
 7: 247,
 8: 214,
 9: 148,
 10: 118,
 11: 86,
 12: 60,
 13: 58,
 14: 57,
 15: 49,
 16: 41,
 17: 39,
 18: 33,
 19: 25,
 20: 22,
 21: 20,
 22: 23,
 23: 21,
 24: 16,
 25: 16,
 26: 20,
 27: 15,
 28: 11,
 29: 13,
 30: 10,
 31: 8,
 32: 5,
 33: 14,
 34: 3,
 35: 7,
 36: 5,
 37: 3,
 38: 5,
 39: 7,
 40: 3,
 41: 3,
 42: 2,
 43: 3,
 44: 8,
 45: 6,
 46: 1,
 47: 7,
 48: 2,
 49: 2,
 50: 5,
 52: 1,
 53: 2,
 54: 2,
 55: 2,
 56: 2,
 57: 2,
 58: 2,
 59: 1,
 60: 1,
 62: 3,
 63: 3,
 64: 4,
 66: 2,
 68: 1,
 69: 1,
 70: 2,
 71: 1,
 72: 1,
 73: 1,
 74: 2,
 76: 2,
 77: 1,
 78: 1,
 79: 2,
 81: 1,
 85: 1,
 86: 1,
 88: 3,
 89: 1,
 90: 1,
 91: 4,
 92: 1,
 93: 2,
 96: 2,
 97: 1,
 98: 2,
 101: 2,
 102: 1,
 105: 1,
 107: 1,
 108: 1,
 112: 1,
 114: 3,
 118: 1,
 120: 1,
 121: 1,
 124: 1,
 125: 2,
 127: 2,
 130: 1,
 137: 1,
 139: 1,
 145: 1,
 147: 1,
 148: 1,
 154: 1,
 167: 1,
 170: 1,
 172: 1,
 184: 1,
 188: 1,
 197: 1,
 218: 1,
 235: 1,
 257: 1,
 270: 1,
 274: 

In [353]:
effectivecount1_10Train={}
freqbetween1_10Train = frequencyOFcountsTrain.values()[:12]

for i in range(0,10):
    p = float((i+2)*freqbetween1_10Train[i+1])/freqbetween1_10Train[i]
    effectivecount1_10Train[i+1]=p
  

In [354]:
effectivecount1_10Train  

{1: 0.3150206618188807,
 2: 1.111653447223917,
 3: 1.9802414928649836,
 4: 2.7993348115299335,
 5: 4.41980198019802,
 6: 4.647849462365591,
 7: 6.931174089068826,
 8: 6.224299065420561,
 9: 7.972972972972973,
 10: 8.016949152542374}

In [358]:
lstdTrain=[]
d_gtTrain=0

for key in effectivecount1_10Train:
    lstdTrain.append(key - effectivecount1_10Train[key])
    
dTrain=np.mean(lstdTrain)

In [359]:
lstdTrain


[0.6849793381811193,
 0.888346552776083,
 1.0197585071350164,
 1.2006651884700665,
 0.5801980198019798,
 1.352150537634409,
 0.0688259109311744,
 1.7757009345794392,
 1.0270270270270272,
 1.9830508474576263]

In [360]:
dTrain

1.0580702863993943

In [362]:
def good_turingTrain(one,two):
    c=0
    #gt_effectivecount=0
    gt_bigramProb=0
    d = unigramDictTrain[one] 
    
    if (one,two) in bigramDictTrain.keys():
        c=bigramDictTrain[(one,two)]
        if bigramDictTrain[(one,two)]==1:
            gt_bigramProb = float(c - lstdTrain[0])/d
        else:            
            gt_bigramProb = float(c - dTrain)/d
    else:
        gt_bigramProb = float(frequencyOFcountsTrain[1])/len(trainwords)

    return gt_bigramProb

In [363]:
good_turingTrain('sherlock','is')

0.33466267863077437

In [364]:
good_turingTrain('that','anyone')

0.0002340420964479054

## Test

In [366]:
test =' '.join(testing)
testwords = test.split(' ')
testwords.remove('')

In [374]:
#testwords

In [375]:
TESTbigramslist=[]

flag=0
for i in range(0,len(testwords)-1):
    if testwords[i]=="</S>" and testwords[i+1]=="<S>":
        flag=1
    else:
        TESTbigramslist.append((testwords[i],testwords[i+1]))

TESTbigramDict = FreqDist(TESTbigramslist)

In [376]:
#TESTbigramslist

### Perplexity for add-one smoothing

In [378]:

probAddOne=0

for i in range(0,len(testwords)-1):
    temp = add_oneTrain(testwords[i],testwords[i+1])
    probAddOne = probAddOne + math.log(temp)

expprobAddOne = math.exp(probAddOne)  
    

In [379]:
probAddOne

-186887.93498732225

### Perplexity for good turing 

In [380]:

probGoodTuring=0

for i in range(0,len(testwords)-1):
    temp = good_turingTrain(testwords[i],testwords[i+1])
    probGoodTuring = probGoodTuring + math.log(temp)
    
expprobGoodTuring = math.exp(probGoodTuring)  


In [381]:
probGoodTuring

-69781.18118438573

In [385]:
def perplexity(denom):
    temp=0
    ppw=0
    N=len(TESTbigramslist)
    ppw = ((-1)*denom)/N
    
    return math.exp(ppw)

###  Comparing perplexity of add-one and good turing

In [386]:
print '\nPerplexity of add-one for the test corpus is : ',perplexity(probAddOne)
print 'Perplexity of good turing for the test corpus is : ',perplexity(probGoodTuring)

print '\nLower the perplexity value for a test corpus, better the model'
print 'Hence in this case good turing performs better'


Perplexity of add-one for the test corpus is :  2726.93971843
Perplexity of good turing for the test corpus is :  19.1791896829

Lower the perplexity value for a test corpus, better the model
Hence in this case good turing performs better
